## Analyzing Used Car Sales

### Project Summary
Here I aim to get at an overview of the automotive auction industry mainly with the objective to figure out where and which model of Ford F150 can be procured at the most affordable price.

**Note:**
Ford F-150 is the highest selling automobile for decades in the US.

### Objectives:
1. Read in and manipulate data with Pandas.
2. Build summary charts with Matplotlib and Seaborn.
3. Leverage Seaborn's advanced chart types to mine insights from the data and make a decision.

### The Data Set:
The data set consists of used car auction data (United States and Canada).
(Source: Kaggle; https://www.kaggle.com/datasets/tunguz/used-car-auction-prices)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Read in the dataset `car_prices.csv`.
# on_bad_lines='skip' is used to avoid data that has inconsistent rows.
cars = pd.read_csv("../Data/car_prices.csv", on_bad_lines='skip')

In [ ]:
cars.head()

In [ ]:
cars.info()

In [ ]:
cars.describe()

# Numeric Variable Relationships

In [ ]:
# Pairplot with upper corner turned off.
sns.pairplot(
    cars,
    corner = True
)
plt.show()

* The relationship between `sellingprice` and `mmr` ('Manheim Market Report' or the recommended selling price) is strong because `mmr` would have been set by experts in the industry who can predict the price at which the car would sell.
* The newer the car, more its selling price (`sellingprice` increases with `year`).

In [ ]:
# Filter dataframe down to `body` styles ['SUV', 'Sedan', 'Convertible', and 'Coupe']. Color the scatterplot based on values in the `body` column. 
style_list = ["SUV", "Sedan", "Convertible", "Coupe"]

g = sns.pairplot(
    cars.query('body in @style_list'),
    corner = True,
    hue = 'body'
)

g.fig.suptitle('Sedans and SUVs Dominate', fontsize=14, x=0.3, y=0.915)
g.fig.subplots_adjust(top=0.9)
plt.show()

#### Correlation Heat Map

In [ ]:
# Create a correlation matrix.
cars_corr = cars.select_dtypes(include=[np.number]).corr()
cars_corr

In [ ]:
# Create a heatmap with the correlation matrix.
sns.heatmap(
    cars_corr,
    cmap = 'RdYlGn',
    annot = True,
    fmt = '0.2f'
)
plt.show()

* `condition` and `year` have negative correlation with `odometer`. This makes sense. Newer the car and better the condition, it would have lesser `odometer` readings (lesser kms it would have driven) and vice-versa.

#### Value Retention

In [ ]:
# Create a data frame with the brands ['Ford', 'BMW', 'Toyota', 'Chevrolet'] filtered.
cars_make = cars.query("make in ['Ford', 'BMW', 'Toyota', 'Chevrolet']")
cars_make.head()

In [ ]:
# Create a linear relationship plot between 'odometer' and 'selling price' for each of the 5 brands.
# Filter down appropriately so that the data points are not very crowded.


g = sns.lmplot(
    x = 'odometer',
    y = 'sellingprice',
    hue = 'make',
    scatter_kws = {'alpha': 0.3},
    data=cars_make.query("sellingprice <150000 and odometer < 200000").sample(1000)
)
sns.despine()
g.fig.suptitle('BMW Price Declines the Most With Mileage', fontsize=12)
g.fig.subplots_adjust(top=0.9)

In [ ]:
# To get the parameters (slope & intercept) of the regression lines of each brand.
import scipy.stats as stats
brand_list = ['Ford', 'BMW', 'Toyota', 'Chevrolet']
cars_regress = cars_make.dropna()

for make in cars_regress['make'].unique():
    filtered_df = cars_regress.loc[cars_regress['make']==make]
    print(f"{make}: {stats.linregress(x=filtered_df['odometer'], y=filtered_df['sellingprice'])[0:2]}")

* Toyota loses the least amount of value with mileage whereas BMW loses the most.

# Categorical Variable Relationships

In [ ]:
# Add a column 'condition_bins' to assign each row into bins with an increment of 0.5 based on the 'condition' variable.
# Build a bar plot of average price by condition.

bins = np.linspace(1, 5, 9)
labels = ["1-1.5", "1.5-2", "2-2.5", "2.5-3", "3-3.5", "3.5-4", "4-4.5", "4.5-5"]
cars = cars.assign(
    condition_bins = pd.cut(cars['condition'], bins = bins, labels = labels)
)

fig, ax = plt.subplots(figsize=(8, 8))
sns.barplot(
    x = 'condition_bins',
    y = 'sellingprice',
    hue = 'condition_bins',
    legend=None,
    data = cars
)

ax.set_title('Average Price by Condition')
ax.set_xlabel('Condition Bins')
ax.set_ylabel('Selling Price')
sns.despine()

# Price by make and condition

In [ ]:
# Find the top 10 most common car brands.
top_10_brands = cars['make'].value_counts().iloc[0:10].index
top_10_brands

In [ ]:
# Create a pivot table with the common car brands as rows, condition bins as columns and average selling price as values.

top_10_brands_pivot = cars.query("make in @top_10_brands").pivot_table(
    index = 'make',
    columns = 'condition_bins',
    values = 'sellingprice',
    aggfunc = 'mean'
)
top_10_brands_pivot

In [ ]:
# Build a heat map from the pivot table above.
fig, ax = plt.subplots(figsize = (8, 8))
sns.heatmap(
    top_10_brands_pivot,
    cmap = 'RdYlGn',
    annot = True,
    fmt = 'g'
)

ax.set_title('Average Selling Price by Make and Quality')
ax.set_xlabel('Condition Bins')
ax.set_ylabel('Brand')
plt.show()

* BMW has the highest selling price in its best condition.
* In its lowest condition, BMW sells at a lesser price than Honda, Toyota, Nissan & Kia.
* Chevrolet and Ford have higher selling prices in its best condition.
* American car brands sell trucks and SUVs more; they have a higher selling price than a Sedan. Toyota sells more Sedans than Ford and Chrysler which explains the increased prices of Ford and Chrysler as compared to Toyota.

# Ford F-150 Deep Dive

In [ ]:
# Filter your data down to rows where 'model' is 'F-150'.
f150s = cars[cars["model"] == "F-150"]

f150s.head()

In [ ]:
# Build a histogram of sellingprice with 10 bins.
fig, ax = plt.subplots()
sns.histplot(
    f150s['sellingprice'],
    bins = 10
)

ax.set_title('Distribution of Selling Price of Ford F150')
ax.set_xlabel('Selling Price')
sns.despine()
plt.show()

In [ ]:
# Build a barplot of `sellingprice` by `trim` for the F150s.
# Car trim refers to a particular version of a model with a particular set of configuration, and is usually designated with a specific moniker.

fig, ax = plt.subplots()
sns.barplot(
    x = 'trim',
    y = 'sellingprice',
    hue = 'trim',
    data = f150s
)

ax.set_title('Selling Price by Car Trims of Ford F150')
ax.set_xlabel('Trim')
ax.set_ylabel('Selling Price')
ax.xaxis.set_tick_params(rotation=90)
sns.despine()

In [ ]:
# Filter the dataset down to the trim levels ["XL", "XLT", "Platinum", "SVT Raptor"] (Current available trim models).
# Then, build a jointplot of `condition` (not binned) by `sellingprice`.

trim_list = ['XL', 'XLT', 'Platinum', 'SVT Raptor']
sns.jointplot(
    x = 'condition',
    y = 'sellingprice',
    hue = 'trim',
    data = f150s.query("trim in @trim_list")
)
plt.show()

* XL has the lowest and SVT raptor has the highest selling price across condition bins.

### Are there any "Deals"?

In [ ]:
# Create a column `diff_to_mmr` which is the difference between `sellingprice` and `mmr`.
# Group by color, aggregating by mean of 'diff_to_mmr'.
f150s_by_color = f150s.assign(
     diff_to_mmr = f150s['sellingprice'] - f150s['mmr']
 ).groupby('color').agg({'diff_to_mmr': 'mean'})
f150s_by_color

In [ ]:
# Build a barplot looking at mean `diff_to_mmr` by `color.
sns.set_style("darkgrid")
fig, ax = plt.subplots()
sns.barplot(
    x = 'color',
    y = 'diff_to_mmr',
    hue = 'color',
    errorbar = None,
    legend = None,
    data = f150s_by_color
)
ax.set_title('Orange Color Sells Above the Recommended Price')
ax.set_xlabel('Color')
ax.set_ylabel('Selling Price - Recommended Price')
ax.xaxis.set_tick_params(rotation=90)

* Orange is the only colour that sells above the recommended retail price. All other colors sell below or at the recommended price.
* Charcoal sells alomost 600 dollars below the recommended price. If color is not a criteria, then going with charcoal color would cost us the least.

In [ ]:
# Further filter the data to the trims above. 
# Exclude the color 'orange' and look only at trucks from years between 2009 and 2014.
trim_list = ['XL', 'XLT', 'Platinum', 'SVT Raptor']
f150s_reduced = f150s.query("trim in @trim_list and color != 'orange'").loc[f150s['year'].between(2009, 2014)]
f150s_reduced.head()

In [ ]:
# Create a pivot table with trim as the index and condition_bins as columns and mean selling price as values.
f150s_pivot = f150s_reduced.pivot_table(
    index = 'trim',
    columns = 'condition_bins',
    values =  'sellingprice',
    aggfunc = 'mean'
)
f150s_pivot

In [ ]:
# Create a heat map with the pivot table.
sns.reset_defaults()
fig, ax = plt.subplots(figsize = (10, 10))
sns.heatmap(
    f150s_pivot,
    annot = True,
    fmt = 'g'
)
plt.show()

* XL and XLT are the cheapest options available. Let's decide to go ahead with XLT as it has more features than XL but is still quite affordable.

### Best state to buy trucks in?

In [ ]:
# Filter out any trucks with a quality less than 3.5 and group them by state, aggregating by mean and count of 'diff_to_mmr'.
f150s_xlr_quality = f150s_reduced.query("condition > 3.5 and trim == 'XLT'").assign(
    diff_to_mmr = f150s_reduced['sellingprice'] - f150s_reduced['mmr']
).groupby('state').agg(
    mean_diff=("diff_to_mmr", 'mean'),
    auto_count=("diff_to_mmr", "count")
)
f150s_xlr_quality.head()

In [ ]:
# Build a subplot with a bar chart of `state` by `diff_to_mmr` and `state` by `count` 
# Figure out which states sell XLT models below the recommended price and have ample quantity.

fig, ax = plt.subplots(2, figsize = (12, 6))
sns.barplot(
    x= 'state',
    y = 'mean_diff',
    hue = 'state',
    legend = None,
    data = f150s_xlr_quality,
    ax = ax[0]
)

sns.barplot(
    x= 'state',
    y = 'auto_count',
    hue = 'state',
    legend = None,
    data = f150s_xlr_quality,
    ax = ax[1]
)
plt.show()

* Alberta, Alabama, New Mexico and Utah have selling price significantly lower than recommended price. Out of these Alberta and Utah have good quantity available.

In [ ]:
# List the mean and count of 'diff_to_mmr' by 'saledate' in Utah.
f150s.query("trim=='XLT' and state == 'ut' and condition > 3.5").assign(diff_to_mmr = f150s['sellingprice'] - f150s['mmr']).groupby("saledate").agg({"diff_to_mmr":["mean", "count"]})

In [ ]:
# List the mean and count of 'diff_to_mmr' by 'saledate' in Alberta.
f150s.query("trim=='XLT' and state == 'ab' and condition > 3.5").assign(diff_to_mmr = f150s['sellingprice'] - f150s['mmr']).groupby("saledate").agg({"diff_to_mmr":["mean", "count"]})

### Conclusion
* Ford F-150 with trim model 'XLT' is popular and affordable. It can be procured from Utah or Alberta at a reasonable price.